In [54]:
import pandas as pd
import json
import numpy as np
import torch
from torch.autograd import Variable
import statsmodels.formula.api as smf
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
with open('order_dict.json') as f:
    data = json.load(f)

In [4]:
order_dict = {int(k):pd.DataFrame(v).set_index('day_of_month') for k, v in data.items()}

In [49]:
order_dict[1]

,demand,day_of_week,order_hour_mode,order_hour_08,order_hour_95,att1_high,att1_miss,att2_high,att2_miss,type_1_percent,...,fast_promise,original_unit_price_mean,discount_rate_mean,direct_discount_rate_mean,quantity_discount_rate_mean,bundle_discount_mean,gift_mean,dc_ori_mode,dc_ori_num,cluster_id
day_of_month,,,,,,,,,,,,,,,,,,,,,
1,5,3,12,0.000000,1.000000,0.750000,0.250000,0.000000,0.750000,0.000000,...,0.250000,121.475000,0.070538,0.061069,0.009470,0.000000,0.000000,7,1,0
1,18,3,5,0.444444,0.555556,1.000000,0.000000,1.000000,0.000000,1.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,7,1,1
1,57,3,23,0.111111,0.400000,0.488889,0.222222,0.333333,0.355556,0.933333,...,0.888889,108.322222,0.468941,0.104816,0.238231,0.000000,0.044444,7,3,2
1,11,3,14,0.090909,0.636364,1.000000,0.000000,0.545455,0.090909,0.090909,...,0.272727,144.081818,0.669827,0.568953,0.027548,0.000000,0.000000,7,1,3
1,58,3,12,0.111111,0.416667,0.777778,0.083333,0.500000,0.111111,0.333333,...,0.472222,125.488889,0.421472,0.170375,0.178280,0.000000,0.000000,7,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,39,5,14,0.028571,0.742857,0.400000,0.342857,0.628571,0.057143,0.057143,...,0.171429,104.162857,0.335590,0.118868,0.003852,0.013378,0.028571,7,2,6
31,16,5,15,0.000000,0.866667,0.733333,0.200000,0.533333,0.200000,0.000000,...,0.000000,1.200000,0.933333,0.000000,0.000000,0.000000,0.933333,7,1,7
31,26,5,13,0.090909,0.636364,0.590909,0.272727,0.454545,0.272727,0.727273,...,0.727273,111.381818,0.412384,0.084126,NaN,0.039330,0.272727,1,3,8


In [166]:
# identify numerical and categorical features
cat = ['cluster_id','dc_ori_mode','day_of_week','order_hour_mode']
num = df.drop(cat+['demand'], axis = 1).columns.to_list()
feature = df.drop('demand', axis = 1).columns.to_list()

In [170]:
summary = pd.DataFrame()
i = 0
for k,v in order_dict.items():
    summary.loc[i, 'destination'] = k
    summary.loc[i, 'num of days'] = v.index.nunique()
    summary.loc[i, 'num of clusters'] = v['cluster_id'].nunique()
    summary.loc[i, 'length'] = len(v)
    
    #print('k:',k)
    for variable in cat:
        train_set = set(v.query('1<= day_of_month <= 24')[variable])
        test_set = set(v.query('24 < day_of_month <= 31')[variable])
        summary.loc[i, str(variable)+' not in train cnt'] = len(test_set.difference(train_set))
        #print(len(test_cluster), len(train_cluster))
    
    i += 1

In [174]:
summary.head()

,destination,num of days,num of clusters,length,cluster_id not in train cnt,dc_ori_mode not in train cnt,day_of_week not in train cnt,order_hour_mode not in train cnt
0,1.0,31.0,11.0,255.0,0.0,0.0,0.0,0.0
1,2.0,31.0,10.0,300.0,0.0,0.0,0.0,0.0
2,3.0,31.0,10.0,251.0,0.0,0.0,0.0,0.0
3,4.0,31.0,10.0,301.0,0.0,0.0,0.0,0.0
4,5.0,31.0,10.0,303.0,0.0,0.0,0.0,0.0


In [173]:
summary['dc_ori_mode not in train cnt'].value_counts() 

0.0    52
1.0     7
2.0     1
Name: dc_ori_mode not in train cnt, dtype: int64

In [176]:
summary['order_hour_mode not in train cnt'].value_counts()

0.0    39
1.0    21
Name: order_hour_mode not in train cnt, dtype: int64

- 8 datasets have the problem that there are 1-2 categories of dc_ori_mode that are in the test set and not in the train set
- 21 datasets have the problem that there is 1 categoriy of order_hour_mode that is in test set and not in the train set

In [50]:
df = order_dict[1].copy()
df = df.fillna(0)
df.replace(np.inf,0, inplace = True)

In [152]:
# convert data type for categorical features
df['cluster_id'] = df['cluster_id'].astype('object')
df['dc_ori_mode'] = df['dc_ori_mode'].astype('object')
df['day_of_week'] = df['day_of_week'].astype('object')
df['order_hour_mode'] = df['order_hour_mode'].astype('object') #??? numerical or categorical
# order_hour_mode??

In [153]:
# train test split
train = df.query('1<= day_of_month <= 24')
test = df.query('24 < day_of_month <= 31')

train_enc = pd.get_dummies(train, columns = cat)
test_enc = pd.get_dummies(test, columns = cat)

x_train = train_enc.drop('demand', axis = 1).to_numpy()
y_train = train_enc['demand'].to_numpy()

x_test = test_enc.drop('demand', axis = 1)
y_test = test_enc['demand']

In [154]:
# check VIF

def VIF(df, columns):
    
    values = sm.add_constant(df[columns]).values
    num_columns = len(columns)+1
    vif = [variance_inflation_factor(values, i) for i in range(num_columns)]
    
    return pd.Series(vif[1:], index=columns)

VIF(train, num).sort_values()

order_hour_08                   1.227447
order_hour_95                   1.243840
att2_high                       1.940151
dc_ori_num                      2.310836
original_unit_price_mean        2.682505
att1_high                       2.701078
att2_miss                       3.409414
att1_miss                       3.977800
no_promise                      4.684262
bundle_discount_mean            5.576874
fast_promise                    6.966579
type_1_percent                  7.833754
quantity_discount_rate_mean     8.270174
direct_discount_rate_mean      17.705138
discount_rate_mean             50.929713
gift_mean                      63.731707
dtype: float64

In [155]:
ols = smf.ols('demand ~'+ '+'.join(df[feature].columns), data = train).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 demand   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     10.75
Date:                Tue, 01 Mar 2022   Prob (F-statistic):           4.30e-30
Time:                        22:29:33   Log-Likelihood:                -661.34
No. Observations:                 198   AIC:                             1437.
Df Residuals:                     141   BIC:                             1624.
Df Model:                          56                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      -5.6440     12.104     -0.466      0.642     -29.573      18.285
day_of_week[T.1]                0.5325      2.550      0.209      0.835      -4.509       5.574
day_of_week[T.2]                2.3608      2.663      0.886      0.377      -2.904       7.625
day_of_week[T.3]                7.6833      2.443      3.145      0.002       2.854      12.513
day_of_week[T.4]                3.0735      2.412      1.274      0.205      -1.696       7.843
day_of_week[T.5]                1.5401      2.498      0.617      0.539      -3.398       6.478
day_of_week[T.6]                0.8550      2.732      0.313      0.755      -4.546       6.256
order_hour_mode[T.3]          -27.1059     14.182     -1.911      0.058     -55.143       0.931
order_hour_mode[T.5]           -7.7332     11.793     -0.656      0.513     -31.047      15.580
order_hour_mode[T.6]           -9.1778     12.448     -0.737      0.462     -33.787      15.431
order_hour_mode[T.7]           -4.8606     13.728     -0.354      0.724     -31.999      22.278
order_hour_mode[T.8]          -17.4247      9.537     -1.827      0.070     -36.279       1.430
order_hour_mode[T.9]          -14.8019      9.766     -1.516      0.132     -34.108       4.505
order_hour_mode[T.10]         -13.3821      9.117     -1.468      0.144     -31.406       4.642
order_hour_mode[T.11]         -14.1795      9.252     -1.533      0.128     -32.470       4.111
order_hour_mode[T.12]         -13.8732      9.038     -1.535      0.127     -31.741       3.994
order_hour_mode[T.13]         -11.3455      9.309     -1.219      0.225     -29.749       7.058
order_hour_mode[T.14]         -16.0616      9.064     -1.772      0.079     -33.980       1.856
order_hour_mode[T.15]         -12.2839      9.125     -1.346      0.180     -30.324       5.756
order_hour_mode[T.16]         -14.5731      9.278     -1.571      0.119     -32.916       3.770
order_hour_mode[T.17]         -10.7899      9.236     -1.168      0.245     -29.050       7.470
order_hour_mode[T.18]         -12.4626      9.435     -1.321      0.189     -31.116       6.191
order_hour_mode[T.19]         -14.3320      9.439     -1.518      0.131     -32.993       4.329
order_hour_mode[T.20]         -15.3280      9.079     -1.688      0.094     -33.276       2.620
order_hour_mode[T.21]         -17.9003      9.257     -1.934      0.055     -36.201       0.401
order_hour_mode[T.22]         -13.6000      9.211     -1.476      0.142     -31.810       4.610
order_hour_mode[T.23]         -16.5132      9.310     -1.774      0.078     -34.919       1.892
dc_ori_mode[T.7]                7.7294      3.335      2.317      0.022       1.136      14.323
dc_ori_mode[T.56]              -8.3660     13.452     -0.622      0.535     -34.961      18.229
dc_ori_mode

In [165]:
ols.params[ols.pvalues < 0.05]

day_of_week[T.3]              7.683284
dc_ori_mode[T.7]              7.729392
cluster_id[T.6]              14.859291
cluster_id[T.7]              18.962823
cluster_id[T.8]              12.578713
cluster_id[T.9]              91.087682
cluster_id[T.10]            -28.009208
discount_rate_mean           71.915166
direct_discount_rate_mean   -58.488660
bundle_discount_mean        -79.581142
gift_mean                   -76.828877
dc_ori_num                    6.298927
dtype: float64

In [157]:
def remove_high_VIF(variable, numerical_feature_list, feature_list):
    # take in a numerical variable name that you want to remove 
    # and a list of numerical features and features
    # print updated VIFs
    # print ols adjusted R-square
    num2 = numerical_feature_list.copy()
    num2.remove(variable)
    feature_list2 = feature_list.copy()
    feature_list2.remove(variable)
    print('removing:', variable)
    print('\n')
    print('updated VIF:')
    print(VIF(train, num2).sort_values())
    ols2 = smf.ols('demand ~'+ '+'.join(df[feature_list].columns), data = train).fit()
    print('\n')
    print('adjusted r-square:', ols2.rsquared_adj)

In [158]:
remove_high_VIF('discount_rate_mean', num, feature)

removing: discount_rate_mean


updated VIF:
order_hour_08                  1.224566
order_hour_95                  1.233925
bundle_discount_mean           1.338781
direct_discount_rate_mean      1.384582
quantity_discount_rate_mean    1.725968
att2_high                      1.939976
dc_ori_num                     2.305661
original_unit_price_mean       2.682463
att1_high                      2.700804
gift_mean                      2.757746
att2_miss                      3.402021
att1_miss                      3.969414
no_promise                     4.675304
fast_promise                   6.965184
type_1_percent                 7.802125
dtype: float64


adjusted r-square: 0.7347821993580925


In [159]:
# test remove gift_mean, not as good as removing discount_rate_mean
remove_high_VIF('gift_mean', num, feature)

removing: gift_mean


updated VIF:
order_hour_08                  1.225468
order_hour_95                  1.239387
bundle_discount_mean           1.357152
quantity_discount_rate_mean    1.497704
direct_discount_rate_mean      1.829491
att2_high                      1.939538
discount_rate_mean             2.203789
dc_ori_num                     2.301590
original_unit_price_mean       2.623259
att1_high                      2.688293
att2_miss                      3.406225
att1_miss                      3.962143
no_promise                     4.673146
fast_promise                   6.965832
type_1_percent                 7.819835
dtype: float64


adjusted r-square: 0.7347821993580925


In [160]:
# remove discount_rate_mean
num2 = num.copy()
num2.remove('discount_rate_mean')
feature2 = feature.copy()
feature2.remove('discount_rate_mean')

In [161]:
# test remove type_1_percent
remove_high_VIF('type_1_percent', num2, feature2)

removing: type_1_percent


updated VIF:
order_hour_08                  1.224462
order_hour_95                  1.224808
bundle_discount_mean           1.307940
direct_discount_rate_mean      1.379373
quantity_discount_rate_mean    1.651688
att2_high                      1.829177
dc_ori_num                     2.123439
original_unit_price_mean       2.645974
att1_high                      2.655360
gift_mean                      2.732152
att2_miss                      3.383906
att1_miss                      3.885025
no_promise                     4.274968
fast_promise                   4.332225
dtype: float64


adjusted r-square: 0.7022334655244704


In [162]:
ols2 = smf.ols('demand ~'+ '+'.join(df[feature2].columns), data = train).fit()
ols2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 demand   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     9.447
Date:                Tue, 01 Mar 2022   Prob (F-statistic):           3.51e-27
Time:                        22:31:21   Log-Likelihood:                -673.50
No. Observations:                 198   AIC:                             1459.
Df Residuals:                     142   BIC:                             1643.
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      -3.9824     12.819     -0.311      0.757     -29.323      21.358
day_of_week[T.1]                0.2857      2.702      0.106      0.916      -5.055       5.626
day_of_week[T.2]                1.5078      2.814      0.536      0.593      -4.055       7.070
day_of_week[T.3]                6.9493      2.582      2.691      0.008       1.845      12.054
day_of_week[T.4]                2.5515      2.553      0.999      0.319      -2.495       7.598
day_of_week[T.5]               -0.5945      2.594     -0.229      0.819      -5.722       4.533
day_of_week[T.6]               -1.6335      2.829     -0.577      0.565      -7.226       3.959
order_hour_mode[T.3]          -24.7367     15.016     -1.647      0.102     -54.420       4.946
order_hour_mode[T.5]           -6.6187     12.492     -0.530      0.597     -31.314      18.076
order_hour_mode[T.6]          -12.6496     13.162     -0.961      0.338     -38.669      13.369
order_hour_mode[T.7]           -8.5443     14.517     -0.589      0.557     -37.242      20.154
order_hour_mode[T.8]          -16.8023     10.104     -1.663      0.099     -36.777       3.172
order_hour_mode[T.9]          -14.5503     10.348     -1.406      0.162     -35.006       5.905
order_hour_mode[T.10]         -11.3605      9.647     -1.178      0.241     -30.432       7.710
order_hour_mode[T.11]         -12.3941      9.793     -1.266      0.208     -31.754       6.965
order_hour_mode[T.12]         -12.3209      9.569     -1.288      0.200     -31.237       6.595
order_hour_mode[T.13]         -10.2899      9.861     -1.044      0.298     -29.782       9.202
order_hour_mode[T.14]         -12.9823      9.573     -1.356      0.177     -31.907       5.943
order_hour_mode[T.15]          -9.7836      9.649     -1.014      0.312     -28.858       9.291
order_hour_mode[T.16]         -12.7887      9.821     -1.302      0.195     -32.204       6.626
order_hour_mode[T.17]          -8.9122      9.776     -0.912      0.363     -28.237      10.413
order_hour_mode[T.18]         -11.5012      9.995     -1.151      0.252     -31.259       8.257
order_hour_mode[T.19]         -11.8062      9.982     -1.183      0.239     -31.539       7.927
order_hour_mode[T.20]         -14.8130      9.619     -1.540      0.126     -33.828       4.202
order_hour_mode[T.21]         -16.2055      9.800     -1.654      0.100     -35.578       3.167
order_hour_mode[T.22]         -13.0888      9.759     -1.341      0.182     -32.381       6.204
order_hour_mode[T.23]         -14.3163      9.850     -1.453      0.148     -33.788       5.155
dc_ori_mode[T.7]               10.4401      3.470      3.009      0.003       3.580      17.300
dc_ori_mode[T.56]              -2.8715     14.189     -0.202      0.840     -30.921      25.178
dc_ori_mode

In [163]:
ols2.params[ols2.pvalues < 0.05]

day_of_week[T.3]                6.949260
dc_ori_mode[T.7]               10.440119
cluster_id[T.6]                15.728849
cluster_id[T.7]                20.102336
cluster_id[T.8]                15.656592
cluster_id[T.9]                92.481860
cluster_id[T.10]              -28.982323
quantity_discount_rate_mean    66.579974
dc_ori_num                      5.760798
dtype: float64

In [147]:
len(ols2.params)

37

In [43]:
# define model structure

class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out

In [46]:
inputDim = len(x_train.columns)     # takes variable 'x' 
outputDim = 1       # takes variable 'y'
learningRate = 0.01 
epochs = 100

model = linearRegression(inputDim, outputDim)
##### For GPU #######
if torch.cuda.is_available():
    model.cuda()

# initialize the loss (Mean Squared Error) and optimization (Stochastic Gradient Descent)
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

In [57]:
# model training

for epoch in range(epochs):
    # Converting inputs and labels to Variable
    if torch.cuda.is_available():
        inputs = Variable(torch.from_numpy(x_train).cuda())
        labels = Variable(torch.from_numpy(y_train).cuda())
    else:
        inputs = Variable(torch.from_numpy(x_train))
        labels = Variable(torch.from_numpy(y_train))

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    outputs = model(inputs.type(torch.FloatTensor))

    # get loss for the predicted output
    loss = criterion(outputs, labels.type(torch.FloatTensor))
    print(loss)
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    print('epoch {}, loss {}'.format(epoch, loss.item()))

tensor(881.9098, grad_fn=<MseLossBackward0>)
epoch 0, loss 881.9097900390625
tensor(35509280., grad_fn=<MseLossBackward0>)
epoch 1, loss 35509280.0
tensor(2.1446e+12, grad_fn=<MseLossBackward0>)
epoch 2, loss 2144607010816.0
tensor(1.2953e+17, grad_fn=<MseLossBackward0>)
epoch 3, loss 1.2952597444532634e+17
tensor(7.8229e+21, grad_fn=<MseLossBackward0>)
epoch 4, loss 7.82287077223277e+21
tensor(4.7247e+26, grad_fn=<MseLossBackward0>)
epoch 5, loss 4.724712170846058e+26
tensor(2.8535e+31, grad_fn=<MseLossBackward0>)
epoch 6, loss 2.853543910700256e+31
tensor(inf, grad_fn=<MseLossBackward0>)
epoch 7, loss inf
tensor(inf, grad_fn=<MseLossBackward0>)
epoch 8, loss inf
tensor(inf, grad_fn=<MseLossBackward0>)
epoch 9, loss inf
tensor(inf, grad_fn=<MseLossBackward0>)
epoch 10, loss inf
tensor(inf, grad_fn=<MseLossBackward0>)
epoch 11, loss inf
tensor(inf, grad_fn=<MseLossBackward0>)
epoch 12, loss inf
tensor(inf, grad_fn=<MseLossBackward0>)
epoch 13, loss inf
tensor(inf, grad_fn=<MseLossBackw